# Calculate the seasonal means of errors in MDM and QDM+ sort predictions
- MDM: Moment Delta Mapping
- Consider the MDM operators L_1 and L_2 which map the mean and mean and stdev of a distribution respectively
- MDM error for operator L_i  = [L_i (Q_initial) ] - Q_final
- In this notebook, Q_inital is the pre-industrial quantile function and Q_final is the End of 21st century quantile function
- These quantile functions are constructed from detrended CESM2 LENS data
- We can similarly caluclate errors for the Gaussian operators G1 and G2 (see paper for more details)
- In this notebook, we compute the seasonal mean of the MDM error for various operators

In [1]:
import numpy as np
import xarray as xr
from distributed import Client
import dask_jobqueue
#import dask
#import daskdataframe as dd
import matplotlib.pyplot as plt
import matplotlib as mtplt
import glob
import netCDF4 as nc
import zarr
import nc_time_axis
#import xskillscore as xs
import cartopy as cart
#%run data_handler.ipynb
#np.set_printoptions(threshold=np.inf)
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from scipy.special import erfinv, erf
##mtplt.use("TkCairo")
import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 

/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def to_daily(ds):
    year = ds.time.dt.year
    day = ds.time.dt.dayofyear

    # assign new coords
    ds = ds.assign_coords(year=("time", year.data), day=("time", day.data))

    # reshape the array to (..., "day", "year")
    return ds.set_index(time=("year", "day")).unstack("time")  

In [3]:
def stacker(x,stackdim_name):
    #Assumes that x has dimensions year and member
    #x = x.stack(stackdim_name=('year','member'),create_index=False)
    x = x.stack({stackdim_name: ('year', 'member')},create_index=False)
    return x

def stack_roll(ds,roll_len,stride_len,stackdim_name):
    #Assumes that ds has dimensions year and member
    ds_stack   = stacker(ds,stackdim_name)
    #print(ds_stack)
    ds_roll    = ds_stack.rolling({stackdim_name:roll_len}, center=True).construct('index', stride=stride_len)
    
    return ds_roll

In [4]:
def mean_adj(ds_stack,ds_mroll):
    ds = (ds_stack - ds_stack.mean(dim='myear')) + ds_mroll
    return ds

def mean_std_adj(ds_stack,ds_mroll,ds_sroll):
    ds = (ds_stack -ds_stack.mean(dim='myear'))*(ds_sroll/ds_stack.std(dim='myear'))\
          + ds_mroll
    return ds

In [5]:
def gauss_quantile(mean,std,quantile):
    qvalue = mean + std * np.sqrt(2) * erfinv(2*quantile-1)
    return qvalue

In [6]:
def altspace(start, step, count, endpoint=False, **kwargs):
   stop = start+(step*count)
   return np.linspace(start, stop, count, endpoint=endpoint, **kwargs)

In [7]:
# #Chicago
LAT        = 41.8781
LON        = (360-87.6298)%360
# # #Bengaluru:
# LAT        = 12.9716
# LON        = 77.5946
######################
chic_lat  = 41.8781
chic_lon  = (360-87.6298)%360
ben_lat   = 12.9716
ben_lon   = 77.5946
########## CONUS ############
#CONUS lat-lon
top       = 50.0 # north lat
left      = -124.7844079+360 # west long
right     = -66.9513812+360 # east long
bottom    =  24.7433195 # south lat
################################
savefigs   = '/global/scratch/users/harsha/savefigs/Feb21/'
cesm2_path = '/global/scratch/users/harsha/LENS/cesm2/tasmax/'
cvals      = '/global/scratch/users/harsha/LENS/cesm2/cvals/'
cvals_det  = '/global/scratch/users/harsha/LENS/cesm2/cvals/detrended/'
loc        = 'Chicago/'
locn       = 'Chicago'
cvals1     = '/global/scratch/users/harsha/LENS/cesm2/tmax_mem'
###########
print('(lat,lon)=',LAT,LON)

(lat,lon)= 41.8781 272.3702


In [8]:
job_extra = ['--qos=cf_lowprio','--account=ac_cumulus'] 
#job_extra =['--qos=lr6_lowprio','--account=ac_cumulus','--constraint=lr6_m192']
#job_extra =['--qos=condo_cumulus_lr6','--account=lr_cumulus','--constraint=lr6_m192']
#job_extra =['--qos=lr_lowprio','--account=ac_cumulus']
cluster = dask_jobqueue.SLURMCluster(queue="cf1", cores=20, walltime='5:00:00', 
                local_directory='/global/scratch/users/harsha/dask_space/', 
                log_directory='/global/scratch/users/harsha/dask_space/', 
                job_extra_directives=job_extra, interface='eth0', memory="192GB") 
client  = Client(cluster) 
cluster.scale_up(3)

In [9]:
cluster

Dashboard: http://10.0.39.5:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.39.5:43555,Workers: 0
Dashboard: http://10.0.39.5:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [10]:
pi_year0 = '1850'
pi_year1 = '1879'
ic_year0 = '2071'
ic_year1 = '2100'
pi_year  = 1865
# eoc_year = 2085 #Central year used for detrending
ic_year  = 2086 #This is the correct year for the rolling window operation to work
doy      = 211 # day_of_year
## Change/Check these parameters before running
season     = 'MAM'
season0    = 'mam'
months     = [3,4,5]
day        =  91     #First day of the season
date       = 'mar'

In [11]:
#To reduce communication between workers, I am manually fusing tasks
#dask.optimization.fuse(ave_width=50)

In [12]:
%%time
ds_ssp  = xr.open_mfdataset(cesm2_path+'ssp370_cesm2tmax.zarr',engine='zarr')
ds_hist = xr.open_mfdataset(cesm2_path+'bhist_cesm2tmax.zarr',engine='zarr')

CPU times: user 7.41 s, sys: 1.17 s, total: 8.58 s
Wall time: 13.7 s


In [13]:
%%time
#t-temp, nw - no-warming, w-warming, ssn= season, g=global
tgpi_ssn   = ds_hist.TREFHTMX.sel(time=ds_hist.time.dt.month.isin(months)).\
         sel(time=slice(pi_year0,pi_year1))
tgeoc_ssn  = ds_ssp.TREFHTMX.sel(time=ds_ssp.time.dt.month.isin(months)).\
         sel(time=slice(ic_year0,ic_year1))

CPU times: user 1.7 s, sys: 66.4 ms, total: 1.77 s
Wall time: 1.78 s


In [14]:
tgpi_ssn   = tgpi_ssn.chunk({'member':100,'lat':40,'lon':40})
tgeoc_ssn  = tgeoc_ssn.chunk({'member':100,'lat':40,'lon':40})
tdgpi_ssn  = to_daily(tgpi_ssn)
tdgeoc_ssn = to_daily(tgeoc_ssn)

/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/xarray/core/dataset.py:4880: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  result = result._unstack_full_reindex(
/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/xarray/core/dataset.py:4880: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

## Detrend the data to avoid aliasing of warming on to standard deviation

In [15]:
pcoeffs_gpi_ssn  = tdgpi_ssn.mean('member').polyfit(dim='year',deg=1)
pcoeffs_geoc_ssn = tdgeoc_ssn.mean('member').polyfit(dim='year',deg=1)
#pcoeffs_geoc_ssn

In [16]:
mgpi_ssn  = pcoeffs_gpi_ssn.polyfit_coefficients.sel(degree=1)
mgeoc_ssn = pcoeffs_geoc_ssn.polyfit_coefficients.sel(degree=1)
mgeoc_ssn 

<xarray.DataArray 'polyfit_coefficients' (lat: 192, lon: 288, day: 92)>
dask.array<getitem, shape=(192, 288, 92), dtype=float64, chunksize=(5, 288, 92), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * day      (day) int64 60 61 62 63 64 65 66 67 ... 145 146 147 148 149 150 151
    degree   int64 1

In [17]:
gpi_trend_ssn  = mgpi_ssn*(tdgpi_ssn['year']-pi_year)
geoc_trend_ssn = mgeoc_ssn*(tdgeoc_ssn['year']-ic_year)
gpi_trend_ssn

<xarray.DataArray (lat: 192, lon: 288, day: 92, year: 30)>
dask.array<mul, shape=(192, 288, 92, 30), dtype=float64, chunksize=(5, 288, 92, 30), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * day      (day) int64 60 61 62 63 64 65 66 67 ... 145 146 147 148 149 150 151
    degree   int64 1
  * year     (year) int64 1850 1851 1852 1853 1854 ... 1875 1876 1877 1878 1879

In [18]:
tgpi_det_ssn  = tdgpi_ssn  - gpi_trend_ssn
tgeoc_det_ssn = tdgeoc_ssn - geoc_trend_ssn
tgpi_det_ssn  = tgpi_det_ssn.chunk({'day':10,'lat':5})
tgeoc_det_ssn = tgeoc_det_ssn.chunk({'day':10,'lat':5})
tgeoc_det_ssn

<xarray.DataArray (lat: 192, lon: 288, member: 100, year: 30, day: 92)>
dask.array<rechunk-merge, shape=(192, 288, 100, 30, 92), dtype=float64, chunksize=(5, 40, 100, 10, 10), chunktype=numpy.ndarray>
Coordinates:
  * year     (year) int64 2071 2072 2073 2074 2075 ... 2096 2097 2098 2099 2100
  * day      (day) int64 60 61 62 63 64 65 66 67 ... 145 146 147 148 149 150 151
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * member   (member) int64 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
    degree   int64 1

## Apply Moment Delta Mapping to detrended data 
- Before applying MDM to the detrended data, we need to aggregate data from the 30 years 1850-1879 along a single coordinate
- We need to do a similar aggregation for the 30-year period 2071-2100
- This is accomplished by using a rolling window centered at 1865 and 2086, repectively
- In order to understand how the rolling window is implmented in xarray, please check : https://docs.xarray.dev/en/stable/generated/xarray.DataArray.roll.html
- The stack_roll function works by first stacking data from the coordinates 'year' and 'member' onto a single coordinate called 'ym'
- We then use xr.roll to apply the rolling window to this new coordinate 

In [19]:
%%time
#Stack member and year into one dimension- 'ym'. Then, roll over this dimension with stride = N_mem.
#This works because of the way xarray stacks two dimensions. window_len =30. 
#I have called the 'window' dimension 'index'.
quants  = np.arange(0,1.01,0.01)
########################
N_mem       = 100 
window_len  = 30
#### July 30 #######
tgpi_detroll_ssn   = stack_roll(tgpi_det_ssn,N_mem*window_len,N_mem,stackdim_name='ym')
tgeoc_detroll_ssn  = stack_roll(tgeoc_det_ssn,N_mem*window_len,N_mem,stackdim_name='ym')
#########
tgpi_detroll0_ssn  = tgpi_detroll_ssn.sel(ym=[15])  
tgeoc_detroll0_ssn = tgeoc_detroll_ssn.sel(ym=[15])  
tgpi_detroll0_ssn

CPU times: user 3.62 s, sys: 176 ms, total: 3.8 s
Wall time: 3.84 s


<xarray.DataArray (lat: 192, lon: 288, day: 92, ym: 1, index: 3000)>
dask.array<getitem, shape=(192, 288, 92, 1, 3000), dtype=float64, chunksize=(5, 40, 10, 1, 3000), chunktype=numpy.ndarray>
Coordinates:
    year     (ym) int64 1865
  * day      (day) int64 60 61 62 63 64 65 66 67 ... 145 146 147 148 149 150 151
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    member   (ym) int64 0
    degree   int64 1
Dimensions without coordinates: ym, index

In [20]:
%%time
############# Save files ########################
tgeoc_detroll_ssnpath  = cvals_det + 'tgeoc_detrended_ssnroll_'+season0+'_'+ic_year0+'_'+ic_year1+'.zarr'
tgpi_detroll_ssnpath   = cvals_det + 'tgpi_detrended_ssnroll_'+season0+'_'+pi_year0+'_'+pi_year1+'.zarr'
# Save eoc detrended rolled array
tgeoc_detroll0_ssn     =  tgeoc_detroll0_ssn.rename('eoc_detrended').assign_coords(ym=('ym',[2086]))
tgpi_detroll0_ssn      =  tgpi_detroll0_ssn.rename('pi_detrended').assign_coords(ym=('ym',[2086]))
#
print(tgpi_detroll_ssnpath, tgeoc_detroll_ssnpath) 
# tgpi_detroll0_ssn.to_dataset().to_zarr(tgpi_detroll_ssnpath,mode='w') 
# tgeoc_detroll0_ssn.to_dataset().to_zarr(tgeoc_detroll_ssnpath,mode='w')

/global/scratch/users/harsha/LENS/cesm2/cvals/detrended/tgpi_detrended_ssnroll_mam_1850_1879.zarr /global/scratch/users/harsha/LENS/cesm2/cvals/detrended/tgeoc_detrended_ssnroll_mam_2071_2100.zarr
CPU times: user 22.3 ms, sys: 5.7 ms, total: 28 ms
Wall time: 25.3 ms


In [21]:
%%time
#Compute the pre-industrial + moments adjusted curves
#First compute rolling means and stdevs and save them
#Open
tgeoc_detroll_ssn  = xr.open_zarr(tgeoc_detroll_ssnpath).eoc_detrended
tgpi_detroll_ssn   = xr.open_zarr(tgpi_detroll_ssnpath).pi_detrended
#
tgeoc_det_mroll_ssn = tgeoc_detroll_ssn.mean(dim='index')
tgeoc_det_sroll_ssn = tgeoc_detroll_ssn.std(dim='index')
#############
tgeoc_det_mroll_ssn

CPU times: user 732 ms, sys: 52.7 ms, total: 785 ms
Wall time: 804 ms


<xarray.DataArray 'eoc_detrended' (lat: 192, lon: 288, day: 92, ym: 1)>
dask.array<mean_agg-aggregate, shape=(192, 288, 92, 1), dtype=float64, chunksize=(5, 40, 10, 1), chunktype=numpy.ndarray>
Coordinates:
  * day      (day) int64 60 61 62 63 64 65 66 67 ... 145 146 147 148 149 150 151
    degree   int64 ...
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    member   (ym) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    year     (ym) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * ym       (ym) int64 2086

In [22]:
#Now prepare the pre-industrial data array in order to apply the moment delta mapping transformations
###Add a dummy dimension called 'ym' to tgpi_detrended after stacking year and member as 'myear'
############ det ###########
tgpi_det_stack_ssn = stacker(tgpi_det_ssn,'myear')
###################################
tgpi_det_stack0_ssn= tgpi_det_stack_ssn.expand_dims(dim={'ym':1}).\
                 assign_coords(ym=('ym',[2086]))
tgpi_det_stack0_ssn

<xarray.DataArray (ym: 1, lat: 192, lon: 288, day: 92, myear: 3000)>
dask.array<broadcast_to, shape=(1, 192, 288, 92, 3000), dtype=float64, chunksize=(1, 5, 40, 10, 1000), chunktype=numpy.ndarray>
Coordinates:
    year     (myear) int64 1850 1850 1850 1850 1850 ... 1879 1879 1879 1879 1879
  * day      (day) int64 60 61 62 63 64 65 66 67 ... 145 146 147 148 149 150 151
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    member   (myear) int64 0 1 2 3 4 5 6 7 8 9 ... 90 91 92 93 94 95 96 97 98 99
    degree   int64 1
  * ym       (ym) int64 2086
Dimensions without coordinates: myear

In [23]:
%%time
#############
tgmc_det_ssn  = mean_adj(tgpi_det_stack0_ssn, tgeoc_det_mroll_ssn)
tgmsc_det_ssn = mean_std_adj(tgpi_det_stack0_ssn, tgeoc_det_mroll_ssn,tgeoc_det_sroll_ssn)
tgmc_det_ssn

CPU times: user 17.9 s, sys: 825 ms, total: 18.7 s
Wall time: 50.3 s


<xarray.DataArray (ym: 1, lat: 192, lon: 288, day: 92, myear: 3000)>
dask.array<add, shape=(1, 192, 288, 92, 3000), dtype=float64, chunksize=(1, 5, 40, 10, 1000), chunktype=numpy.ndarray>
Coordinates:
  * day      (day) int64 60 61 62 63 64 65 66 67 ... 145 146 147 148 149 150 151
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    degree   int64 1
  * ym       (ym) int64 2086
Dimensions without coordinates: myear

### Apply QDM + Sort

In [24]:
%%time
#### Compute quantiles 
quants        = np.arange(0,1.01,0.01)
#
qgpi_det_ssn  = tgpi_detroll0_ssn.chunk(dict(index=-1)).quantile(quants,dim='index',skipna=False)
qgeoc_det_ssn = tgeoc_detroll0_ssn.chunk(dict(index=-1)).quantile(quants,dim='index',skipna=False)
#
qgmc_det_ssn  = tgmc_det_ssn.chunk(dict(myear=-1)).quantile(quants,dim='myear',skipna=False)
qgmsc_det_ssn = tgmsc_det_ssn.chunk(dict(myear=-1)).quantile(quants,dim='myear',skipna=False)
qgpi_det_ssn

CPU times: user 1.87 s, sys: 79.2 ms, total: 1.95 s
Wall time: 1.94 s


<xarray.DataArray 'pi_detrended' (quantile: 101, lat: 192, lon: 288, day: 92,
                                  ym: 1)>
dask.array<transpose, shape=(101, 192, 288, 92, 1), dtype=float64, chunksize=(101, 5, 40, 10, 1), chunktype=numpy.ndarray>
Coordinates:
    year      (ym) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * day       (day) int64 60 61 62 63 64 65 66 ... 145 146 147 148 149 150 151
  * lat       (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon       (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    member    (ym) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * ym        (ym) int64 2086
  * quantile  (quantile) float64 0.0 0.01 0.02 0.03 0.04 ... 0.97 0.98 0.99 1.0

In [25]:
%%time
qdeltas_ssn = qgeoc_det_ssn - qgpi_det_ssn 
qg_qdm_ssn  = qgpi_det_ssn + qdeltas_ssn
qg_qdm_ssn

CPU times: user 1.47 s, sys: 85.9 ms, total: 1.55 s
Wall time: 3.95 s


<xarray.DataArray (quantile: 101, lat: 192, lon: 288, day: 92, ym: 1)>
dask.array<add, shape=(101, 192, 288, 92, 1), dtype=float64, chunksize=(101, 5, 40, 10, 1), chunktype=numpy.ndarray>
Coordinates:
    year      (ym) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * day       (day) int64 60 61 62 63 64 65 66 ... 145 146 147 148 149 150 151
  * lat       (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon       (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    member    (ym) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * ym        (ym) int64 2086
  * quantile  (quantile) float64 0.0 0.01 0.02 0.03 0.04 ... 0.97 0.98 0.99 1.0

In [26]:
# Now, calculate QDM + sort
qg_sqdm_ssn  = xr.apply_ufunc(
    np.sort,
    qg_qdm_ssn,
    input_core_dims=[['quantile']],
    output_core_dims=[['quantile']],
    dask='parallelized'
)
qg_sqdm_ssn

<xarray.DataArray (lat: 192, lon: 288, day: 92, ym: 1, quantile: 101)>
dask.array<transpose, shape=(192, 288, 92, 1, 101), dtype=float64, chunksize=(5, 40, 10, 1, 101), chunktype=numpy.ndarray>
Coordinates:
    year      (ym) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * day       (day) int64 60 61 62 63 64 65 66 ... 145 146 147 148 149 150 151
  * lat       (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon       (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    member    (ym) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * ym        (ym) int64 2086
  * quantile  (quantile) float64 0.0 0.01 0.02 0.03 0.04 ... 0.97 0.98 0.99 1.0

### Compute Gaussian quantile function

In [27]:
########## EOC gaussians ###########
tgeoc_det_mroll0_ssn = tgeoc_det_mroll_ssn.expand_dims(dim={'quantile':101}).\
               assign_coords(quantile=('quantile',quants))
tgeoc_det_sroll0_ssn = tgeoc_det_sroll_ssn.expand_dims(dim={'quantile':101}).\
               assign_coords(quantile=('quantile',quants))
# Create a DataArray for the quantiles
quantiles = xr.DataArray(np.arange(0, 1.01, 0.01), dims='quantile').\
assign_coords(quantile=('quantile',quants))
####### Construct gaussians for model warming ################
qg        = xr.apply_ufunc(erfinv,2*quantiles-1)
#################
qgauss_det_ssn      = tgeoc_det_mroll0_ssn + tgeoc_det_sroll0_ssn * np.sqrt(2) *qg
qgauss_det_ssn.name = 'qgauss_detrended'
qgauss_det_ssn
## gaussm = gaussian with eoc mean only and pre-ind std
qgaussmo_det_ssn      = tgeoc_det_mroll0_ssn + tgpi_det_stack0_ssn.std('myear')* np.sqrt(2) *qg       
qgaussmo_det_ssn.name = 'qgaussmo_detrended'
qgaussmo_det_ssn

<xarray.DataArray 'qgaussmo_detrended' (quantile: 101, lat: 192, lon: 288,
                                        day: 92, ym: 1)>
dask.array<add, shape=(101, 192, 288, 92, 1), dtype=float64, chunksize=(101, 5, 40, 10, 1), chunktype=numpy.ndarray>
Coordinates:
  * day       (day) int64 60 61 62 63 64 65 66 ... 145 146 147 148 149 150 151
    degree    int64 1
  * lat       (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon       (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    member    (ym) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    year      (ym) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * ym        (ym) int64 2086
  * quantile  (quantile) float64 0.0 0.01 0.02 0.03 0.04 ... 0.97 0.98 0.99 1.0

In [28]:
%%time
qgmsc_ssndetpath     = cvals_det+'qgmsc_percentile_det_ssn_'+season0+'_'+ic_year0+'_'+ic_year1+'.zarr'
qgmc_ssndetpath      = cvals_det+'qgmc_percentile_det_ssn_'+season0+'_' +ic_year0+'_'+ic_year1+'.zarr'
#
qgauss_ssndetpath    = cvals_det+'qgauss_percentile_det_ssn_'+season0+'_' +ic_year0+'_'+ic_year1+'.zarr'
qgaussmo_ssndetpath  = cvals_det+'qgaussmo_percentile_det_ssn_'+season0+'_' +ic_year0+'_'+ic_year1+'.zarr'
#
qg_sqdm_ssndepath    = cvals_det+'qg_sqdm_percentile_det_ssn_'+season0 + '_' +ic_year0 + '_' + ic_year1 +'.zarr'
#
qgeoc_ssndetpath     = cvals_det+'qgeoc_percentile_det_ssn_'+season0+'_' +ic_year0+'_'+ic_year1+'.zarr'

print(qgmc_ssndetpath)
##################### Save #####################
qgmc_det_ssn    = qgmc_det_ssn.rename('qgmc_detrended').chunk({'lat':8})
qgmsc_det_ssn   = qgmsc_det_ssn.rename('qgmsc_detrended').chunk({'lat':8})

qgeoc_det_ssn   = qgeoc_det_ssn.rename('qgeoc_detrended').chunk({'lat':8})
qg_sqdm_ssn     = qg_sqdm_ssn.rename('qsqdm_detrended').chunk({'lat':8})

/global/scratch/users/harsha/LENS/cesm2/cvals/detrended/qgmc_percentile_det_ssn_mam_2071_2100.zarr
CPU times: user 2.29 s, sys: 121 ms, total: 2.41 s
Wall time: 2.43 s


In [ ]:
%%time
#Save files
# qgauss_det_ssn.to_dataset().to_zarr(qgauss_ssndetpath,mode='w')
# qgmc_det_ssn.to_dataset().to_zarr(qgmc_ssndetpath,mode='w') 
# qgmsc_det_ssn.to_dataset().to_zarr(qgmsc_ssndetpath,mode='w')
# qgeoc_det_ssn.to_dataset().to_zarr(qgeoc_ssndetpath,mode='w') 
# qgaussmo_det_ssn.to_dataset().to_zarr(qgaussmo_ssndetpath,mode='w') 
#
qg_sqdm_ssn.to_dataset().to_zarr(qg_sqdm_ssndepath,mode='w') 

/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/distributed/client.py:3125: UserWarning: Sending large graph of size 22.90 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [ ]:
# qgmc_det_ssn     = xr.open_zarr(qgmc_ssndetpath).qgmc_detrended
qgmsc_det_ssn    = xr.open_zarr(qgmsc_ssndetpath).qgmsc_detrended
#
# qgauss_det_ssn   = xr.open_zarr(qgauss_ssndetpath).qgauss_detrended
# qgeoc_det_ssn    = xr.open_zarr(qgeoc_ssndetpath).qgeoc_detrended
# qgaussmo_det_ssn = xr.open_zarr(qgaussmo_ssndetpath).qgaussmo_detrended
#
qg_sqdm_ssn      = xr.open_zarr(qg_sqdm_ssndepath).qsqdm_detrended

In [ ]:
###### Error in prediction for EOC #######
qgmsc_detssn_err    = qgmsc_det_ssn.sel(ym=2086)  - qgeoc_det_ssn.sel(ym=2086)
qgmc_detssn_err     = qgmc_det_ssn.sel(ym=2086)   - qgeoc_det_ssn.sel(ym=2086)
#
qgauss_detssn_err   = qgauss_det_ssn.sel(ym=2086) - qgeoc_det_ssn.sel(ym=2086)
qgaussmo_detssn_err = qgaussmo_det_ssn.sel(ym=2086) - qgeoc_det_ssn.sel(ym=2086)
#
qg_sqdm_ssn_err     = qg_sqdm_ssn.sel(ym=2086) - qgeoc_det_ssn.sel(ym=2086)
qg_sqdm_ssn_err 

In [ ]:
%%time
########### Save the absolute errors ############
####
qgmsc_detssn_errpath    = cvals_det+'qgmsc_detssn_err_' +season0 +'_'+ic_year0+'_'+ic_year1+'.zarr'
qgmc_detssn_errpath     = cvals_det+'qgmc_detssn_err_'  +season0 +'_'+ic_year0+'_'+ic_year1+'.zarr'
qgauss_detssn_errpath   = cvals_det+'qgauss_detssn_err_'+season0 +'_'+ic_year0+'_'+ic_year1+'.zarr'
qgaussmo_detssn_errpath = cvals_det+'qgaussmo_detssn_err_'+season0 +'_'+ic_year0+'_'+ic_year1+'.zarr'
qg_sqdm_ssn_errpath     = cvals_det+ 'qg_sqdm_err_'+season0 +'_'+ic_year0+'_'+ic_year1+'.zarr'
#### Save ########
print(qgmc_detssn_errpath)
qgmsc_detssn_err    = qgmsc_detssn_err.rename('Errors').chunk({'lat':20})
qgmc_detssn_err     = qgmc_detssn_err.rename('Errors').chunk({'lat':20})
qgauss_detssn_err   = qgauss_detssn_err.rename('Errors').chunk({'lat':20})
qgaussmo_detssn_err = qgaussmo_detssn_err.rename('Errors').chunk({'lat':20})
qg_sqdm_ssn_err     = qg_sqdm_ssn_err.rename('Errors').chunk({'lat':20})

############
# qgmsc_detssn_err.to_dataset().to_zarr(qgmsc_detssn_errpath,mode='w')
# qgmc_detssn_err.to_dataset().to_zarr(qgmc_detssn_errpath,mode='w')
# qgauss_detssn_err.to_dataset().to_zarr(qgauss_detssn_errpath,mode='w')
# qgaussmo_detssn_err.to_dataset().to_zarr(qgaussmo_detssn_errpath,mode='w')
qg_sqdm_ssn_err.to_dataset().to_zarr(qg_sqdm_ssn_errpath, mode='w')

In [ ]:
# Open file and plot
qgmc_detssn_err     = xr.open_zarr(qgmc_detssn_errpath).Errors
qgmsc_detssn_err    = xr.open_zarr(qgmsc_detssn_errpath).Errors
qg_sqdm_ssn_err     = xr.open_zarr(qg_sqdm_ssn_errpath).Errors
#
qgaussmo_detssn_err = xr.open_zarr(qgaussmo_detssn_errpath).Errors
qgauss_detssn_err   = xr.open_zarr(qgauss_detssn_errpath).Errors

In [ ]:
######## New colorbar only for shift + stretch and gaussian ############
x = 5
# create a figure and axis
fig, ax = plt.subplots(figsize=(6, 1))
fig.subplots_adjust(bottom=0.5)
# Define colormap
cmap = plt.get_cmap('RdBu_r')
# Make a norm object with the center at 0: TwoSlopeNorm
norm = mcolors.TwoSlopeNorm(vmin=-x, vcenter=0, vmax=x)
# Making numpy array from -3 to 3, with step 0.2
values = np.arange(-x, x+0.25, 0.25)
# Creating a mappable object and setting the norm and cmap for colorbar
mappable = cm.ScalarMappable(norm=norm, cmap=cmap)
mappable.set_array([])
# Creating a colorbar
ticks1 = altspace(-4,1,9)
cbar = plt.colorbar(mappable, ax=ax, orientation='vertical',ticks=ticks1)
cbar.set_label('')
plt.gca().set_visible(False)

In [ ]:
%%time
# Creating a figure and axes
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,5), subplot_kw={'projection': cart.crs.PlateCarree()}\
                       , gridspec_kw = {'wspace':0.05, 'hspace':0.2})

# Plotting qgmsc_err
qgmsc_detssn_err.sel(quantile=0.9).mean('day').plot(ax=axs[0], transform=cart.crs.PlateCarree(),\
                                                    add_colorbar=False, cmap=cmap, norm=norm)
axs[0].coastlines(color="black")
axs[0].set_title(season+r': $L_2(Q_i) - Q_f$ at $p=0.9$')

#Plot QDM + sort
im = qg_sqdm_ssn_err.sel(quantile=0.9).mean('day').plot(ax=axs[1], transform=cart.crs.PlateCarree(),\
                                                   add_colorbar=False, cmap=cmap, norm=norm)
axs[1].coastlines(color="black")
axs[1].set_title(season+r': QDM + sort at $p=0.9$')

# Adding colorbar
cbar = plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.5, orientation='vertical')
cbar.set_label('Error')

plt.show()

In [ ]:
%%time
# Creating a figure and axes
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,5), subplot_kw={'projection': cart.crs.PlateCarree()}\
                       , gridspec_kw = {'wspace':0.05, 'hspace':0.2})

# Plotting qgmsc_err
qgmsc_detssn_err.sel(quantile=0.9).mean('day').plot(ax=axs[0], transform=cart.crs.PlateCarree(),\
                                                    add_colorbar=False, cmap=cmap, norm=norm)
axs[0].coastlines(color="black")
axs[0].set_title(season+r': $L_2(Q_i) - Q_f$ at $p=0.9$')

# Plotting G2, gaussian error
im = qgauss_detssn_err.sel(quantile=0.9).mean('day').plot(ax=axs[1], transform=cart.crs.PlateCarree(),\
                                                          add_colorbar=False, cmap=cmap, norm=norm)
axs[1].coastlines(color="black")
axs[1].set_title(season+': $G_2(Q_i) - Q_f$ at $p=0.9$')

# Adding colorbar
cbar = plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.5, orientation='vertical')
cbar.set_label('Error')

plt.show()

In [ ]:
%%time
# Creating a figure and axes
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,5), subplot_kw={'projection': cart.crs.PlateCarree()}\
                       , gridspec_kw = {'wspace':0.05, 'hspace':0.2})

# Plotting qgmsc_err
qgmc_detssn_err.sel(quantile=0.9).mean('day').plot(ax=axs[0], transform=cart.crs.PlateCarree(),\
                                                    add_colorbar=False, cmap=cmap, norm=norm)
axs[0].coastlines(color="black")
axs[0].set_title(season+': $L_1(Q_i) - Q_f$ at $p=0.9$')

# Plotting da2
im = qgaussmo_detssn_err.sel(quantile=0.9).mean('day').plot(ax=axs[1], transform=cart.crs.PlateCarree(),\
                                                          add_colorbar=False, cmap=cmap, norm=norm)
axs[1].coastlines(color="black")
axs[1].set_title(season+r': $G_1(Q_i) - Q_f$ at $p=0.9$')

# Adding colorbar
cbar = plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.5, orientation='vertical')
cbar.set_label('Errors')

plt.show()

In [ ]:
####### Coded until here ###########